In [1]:
import pandas as pd
import random

In [2]:
# Veut dire que c’est un passage par adresse des 3 variables IX, IY et IZ.
# 3 variables entières qu’il faut initialiser avant le 1er appel de la fonction.
# Veut dire que la fonction est typée, elle retourne un nombre réel.
def alea(IX, IY, IZ):
    # Déclaration d’une variable inter de type réel
    inter = 0.0
    # A chaque exécution de la fonction, les valeurs des 3 variables sont recalculées, d’où un passage par adresse.
    # % : division entière
    # // : reste de la division entière
    IX = 171 * (IX % 177) - 2 * (IX // 177)
    IY = 172 * (IY % 176) - 35 * (IY // 176)
    IZ = 170 * (IZ % 178) - 63 * (IZ // 178)
    # S’il y a un dépassement de capacité sur une variable, on ajoute un nombre très grand, pour la machine 16 bits,
    # pour ramener à une valeur positive. Ce qui marche sur une machine 16 bits marche sur une 32 ou 64 bits.
    if IX < 0:
        IX = IX + 30269
    if IY < 0:
        IY = IY + 30307
    if IZ < 0:
        IZ = IZ + 30323
    inter = ((IX / 30269) + (IY / 30307) + (IZ / 30323))
    return [round(inter - int(inter), 4),IX, IY, IZ]

In [3]:
salaire = [(40000, 30000), (30000, 20000), (20000, 15000), (15000, 10000), (10000, 7500), (7500, 5000), (5000, 3000)]
frequenceSalaire = [5, 10, 10, 15, 20, 20, 20]
age = [(62, 53), (52, 41), (40, 31), (30, 21)]
frequenceAge = [30, 30, 20, 20]
ageEmbauche = [(21, 22), (23, 24), (25, 28), (29, 32), (33, 35), (36, 40)]
frequenceAgeEmbauche = [5, 20, 25, 25, 20, 5]
salaire_embauche=[(32000,24000),(24000,16000),(16000,12000),(12000,8000),(8000,6000),(6000,4000),(4000,3000)]
frequenceSalaireEmbauche=[5,5,10,20,20,20,20]
totEmp=10000
totRet=1000

In [4]:
def cumuleFrequenceIntervale(frequences):
    cumules = []
    index = 0
    for i in frequences:
        cumules.append([index, i+index])
        index += i
    return cumules

In [5]:
def indexFromTable(cumules, aleaNumber):
    i = 0
    for intervale in cumules :
        if intervale[1] > aleaNumber*100 :
            return i
        i+=1
    return i

In [6]:
Employe = pd.DataFrame(columns=["id", "age", "salaire"])
Retraite = pd.DataFrame(columns=["id", "age", "salaire"])

In [7]:
for index in range(11000):
    if index == 0 :
        IX = 100
        IY = 100
        IZ = 100
    else:
        IX = alea(IX, IY, IZ)[1]
        IY = alea(IX, IY, IZ)[2]
        IZ = alea(IX, IY, IZ)[3]
    salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaire), alea(IX, IY, IZ)[0])
    ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAge), alea(IX, IY, IZ)[0])
    i = index+1
    s = random.randrange(salaire[salaireIndex][1], salaire[salaireIndex][0], 100)
    if index <10000:
        a = random.randrange(age[ageIndex][1], age[ageIndex][0], 1)
        new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
        Employe = pd.concat([Employe, new_row], ignore_index=True)
    else:
        a = random.randint(66, 85)
        new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
        Retraite = pd.concat([Retraite, new_row], ignore_index=True)

In [8]:
Employe.head()

,id,age,salaire
0,1,38,6600
1,2,43,8000
2,3,53,29800
3,4,24,3600
4,5,24,3000


In [9]:
Retraite.head()

,id,age,salaire
0,10001,79,7200
1,10002,75,4100
2,10003,69,20500
3,10004,78,15000
4,10005,72,7300


In [10]:
def pensionnaireFive() :
    pensionnaire65 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    for index, row in Retraite.iterrows():
        a = row["age"]
        s = row["salaire"]
        p = ((44*2)/100)*row["salaire"]
        new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
        pensionnaire65 = pd.concat([pensionnaire65, new_row], ignore_index=True)
    return pensionnaire65

In [11]:
pensionnaireFive().head()

,id,age,salaire,pension
0,10001,79,7200,6336.0
1,10002,75,4100,3608.0
2,10003,69,20500,18040.0
3,10004,78,15000,13200.0
4,10005,72,7300,6424.0


In [12]:
def pensionnaireTwo():
    pensionnaire62 = pd.DataFrame(columns=["id", "age", "salaire", "pension"])
    pensionnaire65 = pensionnaireFive()
    pensionnaire62 = pd.concat([pensionnaire62, pensionnaire65], ignore_index=True)
    for index, row in Employe.iterrows():
        if row["age"]>62:
            a = row["age"]
            s = row["salaire"]
            p = ((44*2)/100)*row["salaire"]
            new_row = pd.DataFrame( {"id" : row["id"], "age": [a], "salaire": [s], "pension": [p]})
            pensionnaire65 = pd.concat([pensionnaire65, new_row], ignore_index=True)
    return pensionnaire62

In [13]:
pensionnaireFive().tail()

,id,age,salaire,pension
995,10996,69,8900,7832.0
996,10997,70,36400,32032.0
997,10998,77,3700,3256.0
998,10999,72,8800,7744.0
999,11000,81,26000,22880.0


In [14]:
def newRecru():
    Nouveux_Recrus = pd.DataFrame(columns=["id", "age", "salaire"])
    IX = 100
    IY = 100
    IZ = 100
    a=alea(IX, IY, IZ)[0]
    n = ((400 - 250) * a) + 250
    for index in range(int(n)):
        if index == 0 :
            IX = 100
            IY = 100
            IZ = 100
        else:
            IX = alea(IX, IY, IZ)[1]
            IY = alea(IX, IY, IZ)[2]
            IZ = alea(IX, IY, IZ)[3]
        salaireIndex = indexFromTable(cumuleFrequenceIntervale(frequenceSalaireEmbauche), alea(IX, IY, IZ)[0])
        ageIndex = indexFromTable(cumuleFrequenceIntervale(frequenceAgeEmbauche), alea(IX, IY, IZ)[0])
        i = index+1
        s = random.randrange(salaire_embauche[salaireIndex][1], salaire_embauche[salaireIndex][0], 100)
        a = random.randrange(ageEmbauche[ageIndex][0], ageEmbauche[ageIndex][1], 1)
        new_row = pd.DataFrame( {"id" : [i], "age": [a], "salaire": [s]})
        Nouveux_Recrus = pd.concat([Nouveux_Recrus, new_row], ignore_index=True)
    return Nouveux_Recrus

In [15]:
newRecru().head()

,id,age,salaire
0,1,30,4600
1,2,30,6600
2,3,23,15500
3,4,36,3200
4,5,34,3500
